In [ ]:
#Importing necessary libararies

import os
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from pylab import *
import numpy as np
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
np.random.seed(9876789)
import scipy
from scipy.stats import anderson
from statsmodels.tsa.deterministic import DeterministicProcess

In [ ]:
data=pd.read_csv('../input/coviddatarevised2/covid_data.csv')

In [ ]:
data.head(20)

In [ ]:
data.tail(20)

In [ ]:
#Addinf recr column

In [ ]:
recr=[]
total_cured=0
total_confirmed=0
for i in range(0,len(data)):
    total_cured+=data['cured'][i]
    total_confirmed+=data['confirmed'][i]
    recr.append(total_cured/total_confirmed)


In [ ]:
data['recr']=recr

In [ ]:
data.head(40)

In [ ]:
data.iloc[20:80]

In [ ]:
data.tail(20)

In [ ]:
data[20:150]


In [ ]:
data['recr'].max()

In [ ]:
pd.to_datetime(data['date'])

In [ ]:
data.info()

In [ ]:
data.set_index('date',drop=True,inplace=True)

In [ ]:
data.info()

In [ ]:
index= data.index
index

In [ ]:
data.plot(figsize=(14,6))

In [ ]:
#Preventing overlapping
#Plotting the graph for the last two months
figure,axis= plt.subplots(figsize=(31,6))
figure.autofmt_xdate()
indexr=data['confirmed'].loc['05-03-2021':'05-07-2021'].index
confirmed=data['confirmed'].loc['05-03-2021':'05-07-2021']
axis.plot(indexr,confirmed)
figure.suptitle('Second covid wave')

In [ ]:
data['confirmed:10 days rolling']=data['confirmed'].rolling(window=10,min_periods=1).mean()

In [ ]:
#Simple moving average
data[['confirmed','confirmed:10 days rolling']].plot(figsize=(15,6))

In [ ]:
#Cumulative moving average
data['confirmed'].expanding().mean().plot(figsize=(14,6))

In [ ]:
#Forecasting
#Linear Regression with time series

In [ ]:
y = data.copy()  

#Instantiating`DeterministicProcess` with arguments appropriate for a quintic trend model
dp = DeterministicProcess(

     index=y.index,
  
     order=5,
     drop=True,
    
)

# Creating the feature set for the dates given in y.index
X = dp.in_sample()
X.head()

#  Create features for a 30-day forecast.
X_fore = dp.out_of_sample(steps=20)



In [ ]:
model = LinearRegression()
model.fit(X, y['confirmed'])

y_pred = pd.Series(model.predict(X), index=X.index)
y_fore = pd.Series(model.predict(X_fore), index=X_fore.index)

In [ ]:
ax = y.plot(alpha=0.5, title="Covid cases", ylabel="Confirmed cases",figsize=(14,6))
ax = y_pred.plot(ax=ax, linewidth=3, label="Trend", color='blue')
ax = y_fore.plot(ax=ax, linewidth=3, label="Trend Forecast", color='red')
ax.legend();

In [ ]:
model = LinearRegression()
model.fit(X, y['cured'])

y_pred = pd.Series(model.predict(X), index=X.index)
y_fore = pd.Series(model.predict(X_fore), index=X_fore.index)

In [ ]:
ax = y.plot(alpha=0.5, title="Covid cases-recovery", ylabel="Cured cases",figsize=(14,6))
ax = y_pred.plot(ax=ax, linewidth=3, label="Trend", color='green')
ax = y_fore.plot(ax=ax, linewidth=3, label="Trend Forecast", color='red',ls='--')
ax.legend();



In [ ]:
model = LinearRegression()
model.fit(X, y['cfr'])

y_pred = pd.Series(model.predict(X), index=X.index)
y_fore = pd.Series(model.predict(X_fore), index=X_fore.index)

In [ ]:
ax = y.plot(alpha=0.5, title="Covid cases-cfr", ylabel="Cfr ",figsize=(14,6))
ax = y_pred.plot(ax=ax, linewidth=2, label="Trend", color='black')
ax = y_fore.plot(ax=ax, linewidth=2, label="Trend Forecast", color='red')
ax.legend();

In [ ]:
model = LinearRegression()
model.fit(X, y['recr'])

y_pred = pd.Series(model.predict(X), index=X.index)
y_fore = pd.Series(model.predict(X_fore), index=X_fore.index)

In [ ]:
ax = y.plot(alpha=0.5, title="Covid cases- recr", ylabel="recr",figsize=(14,6))
ax = y_pred.plot(ax=ax, linewidth=2, label="Trend", color='magenta')
ax = y_fore.plot(ax=ax, linewidth=2, label="Trend Forecast", color='red')
ax.legend();

In [ ]:
#The above the graphs show an estimate of the long term trends adhering to linear regression model.
#however using complex/hybrid models would give us more accurate results 

**IMPLEMENTING ARIMA MODEL FROM HERE**


In [ ]:
#Using ARIMA Model

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
def adf_test(series):
    result=adfuller(series)
    print('ADF Statistics: {}'.format(result[0]))
    print('p- value: {}'.format(result[1]))
    if result[1] <= 0.05:
        print(" Data is stationary, null hypothesis rejected")
    else:
        print("Data is non-stationary, null hypothesis accepted ")

In [ ]:
adf_test(data['confirmed'])

In [ ]:
# No need of differencing using logtime series
#To be contd...

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf

In [ ]:
acf = plot_acf(data["confirmed"].dropna())

Taking q value as 27

In [ ]:
pacf = plot_pacf(data["confirmed"].dropna())

Taking p value as 22

If it does not work out then p=10


In [ ]:
data.tail(20)

In [ ]:
data.info()

In [ ]:
from datetime import datetime,timedelta,date
from dateutil import parser



In [ ]:
 from sklearn.model_selection import train_test_split

In [ ]:
#train_data,test_data= train_test_split(data_log_minus_ma,test_size=0.33,random_state=1)

In [ ]:
train_data=data[:'10-06-2021']
test_data=data['10-06-2021':'10-08-2021']

In [ ]:
##prediction
pred_start_date=test_data.index[0]
pred_end_date=test_data.index[-1]
print(pred_start_date)
print(pred_end_date)
print(type(pred_start_date))

In [ ]:
test_data.info()

In [ ]:
#Creating ARIMA Model
from statsmodels.tsa.arima.model import ARIMA
#The plot starts at leg 0(correlation of the data element with itself), so we ignore it 
train_data.info()

In [ ]:
model_ARIMA=ARIMA(train_data['confirmed'],order=(1,1,2))

In [ ]:
model_Arima_fit=model_ARIMA.fit()

In [ ]:
model_Arima_fit.summary()

In [ ]:
test_data['confirmed'].plot(figsize=(15,6))

In [ ]:
test_data['confirmed'].max()

In [ ]:
test_data['confirmed']

In [ ]:
model_Arima_fit.resid.plot(kind='kde')

In [ ]:
!pip install pmdarima

In [ ]:
import pmdarima as pm

In [ ]:

model = pm.auto_arima(train_data['confirmed'], trace=True, error_action='ignore', suppress_warnings=True)
model.fit(train_data['confirmed'])


In [ ]:
forecast = model.predict(n_periods=len(test_data))
forecast
forecast = pd.DataFrame(forecast,index = test_data.index,columns=['confirmed'])

#plot the predictions for validation set
figure,axis= plt.subplots(figsize=(15,6))
plt.plot(train_data['confirmed'], label='Train')
plt.plot(test_data['confirmed'], label='Test')
plt.plot(forecast,label='Prediction')
plt.show()

In [ ]:
forecast

In [ ]:
test_data.index

In [ ]:
fc = model_Arima_fit.predict(test_data.shape[0]) 
fc

In [ ]:
fc = pd.DataFrame(fc,index = test_data.index,columns=['confirmed'])
fc


In [ ]:
fc_series = pd.Series(fc, index=test_data.index)


In [ ]:
plt.figure(figsize=(15,5), dpi=100)
plt.plot(train_data['confirmed'], label='training')
plt.plot(test_data['confirmed'], label='testing')
plt.plot(fc_series, label='forecast')
plt.title('Forecast vs Test data')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
##prediction
pred_start_date=datetime(5,5,2021)
pred_end_date=datetime(10,8,2021)
print(pred_start_date)
print(pred_end_date)

In [ ]:
print(type(pred_start_date))

In [ ]:
print(type(train_data.index))

In [ ]:
test_data.dropna(inplace=True)

In [ ]:
test_data

In [ ]:
pred_values=pd.Series(model_Arima_fit.fittedvalues,copy=True)


In [ ]:
pred_values

In [ ]:
pred_cumsum=pred_values.cumsum()

In [ ]:
pred_log=pd.Series(test_data['confirmed'],index=test_data.index)
pred_log=pred_log.add(pred_cumsum,fill_value=0)

In [ ]:
pred_log

In [ ]:
pred_Arima=np.exp(pred_log)
plt.plot(test_data['confirmed'],color='red')
plt.plot(pred_Arima,color='blue')

In [ ]:
residuals=test_data['confirmed']-pred

In [ ]:
residuals

In [ ]:
model_Arima_fit.resid.plot(kind='kde')

In [ ]:
test_data['Predicted_ARIMA']=pred

In [ ]:
test_data[['confirmed','Predicted_ARIMA']].plot()

In [ ]:

model_Arima_fit.resid.plot(kind='kde')